In [1]:
from nltk.translate import bleu
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from modelT5 import T5Model
import os
import argparse
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import numpy as np
from datasets import Dataset
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq

[nltk_data] Downloading package punkt to /Users/tyler/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/anaconda3/envs/cs-7643/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_test = pd.read_csv('data/COMBINED/test.tsv', sep = '\t', names=['in', 'expected'])
dataset_test = Dataset.from_pandas(df_test)
model = TFAutoModelForSeq2SeqLM.from_pretrained('data/t5savedv4')

2022-04-27 22:32:32.188191: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-27 22:32:32.203597: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at data/t5savedv4.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [3]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def preprocess_function(examples):
    prefix = 'paraphrase: '
    inputs = [prefix + doc for doc in examples["in"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["expected"], max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")
tokenized_test = dataset_test.map(preprocess_function, batched=True)
tokenized_test_small = tokenized_test.shuffle(seed=42).select(range(1500))
tf_test_set = tokenized_test_small.to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

100%|██████████| 6/6 [00:00<00:00, 13.94ba/s]


In [4]:
df_test['sentence_inputs'] = df_test.apply(lambda x: x['in'].split(), axis=1)
df_test['to_model'] = df_test.apply(lambda x: 'paraphrase: ' + x['in'], axis=1)
df_test['encoding'] = df_test.apply(lambda x: tokenizer(x['in'], return_tensors="tf", truncation=True), axis=1)
df_test['input_ids'] = df_test.apply(lambda x: x['encoding']['input_ids'], axis=1)
df_test['attention_mask'] = df_test.apply(lambda x: x['encoding']['attention_mask'], axis=1)
# def model_func(x):
#     return model.generate(
#         input_ids=x['encoding']['input_ids'],
#         do_sample=True,
#         attention_mask=x['encoding']['attention_mask'],
#         max_length=512, top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=2)
# df_test['out'] = df_test.apply(lambda x: model_func(x), axis=1)
df_test.head()

,in,expected,sentence_inputs,to_model,encoding,input_ids,attention_mask
0,"PCCW's chief operating officer, Mike Butcher, ...",Current Chief Operating Officer Mike Butcher a...,"[PCCW's, chief, operating, officer,, Mike, But...","paraphrase: PCCW's chief operating officer, Mi...","[input_ids, attention_mask]","((tf.Tensor(2104, shape=(), dtype=int32), tf.T...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
1,The world's two largest automakers said their ...,Domestic sales at both GM and No. 2 Ford Motor...,"[The, world's, two, largest, automakers, said,...",paraphrase: The world's two largest automakers...,"[input_ids, attention_mask]","((tf.Tensor(37, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...,"[According, to, the, federal, Centers, for, Di...",paraphrase: According to the federal Centers f...,"[input_ids, attention_mask]","((tf.Tensor(2150, shape=(), dtype=int32), tf.T...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...,"[A, tropical, storm, rapidly, developed, in, t...",paraphrase: A tropical storm rapidly developed...,"[input_ids, attention_mask]","((tf.Tensor(71, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."
4,The company didn't detail the costs of the rep...,But company officials expect the costs of the ...,"[The, company, didn't, detail, the, costs, of,...",paraphrase: The company didn't detail the cost...,"[input_ids, attention_mask]","((tf.Tensor(37, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens..."


In [5]:
examples_used = 500
preds_returned = 2
full_outs = []
inps = tokenizer(df_test['in'].values.tolist()[:500],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape

TensorShape([1000, 61])

In [6]:
inps = tokenizer(df_test['in'].values.tolist()[500:1000],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape

TensorShape([1000, 73])

In [7]:
inps = tokenizer(df_test['in'].values.tolist()[1000:1500],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 86]), 3)

In [8]:
inps = tokenizer(df_test['in'].values.tolist()[1500:2000],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 115]), 4)

In [9]:
inps = tokenizer(df_test['in'].values.tolist()[2000:2500],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 68]), 5)

In [10]:
inps = tokenizer(df_test['in'].values.tolist()[2500:3000],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 76]), 6)

In [11]:
inps = tokenizer(df_test['in'].values.tolist()[3000:3500],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 74]), 7)

In [12]:
inps = tokenizer(df_test['in'].values.tolist()[3500:4000],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 69]), 8)

In [13]:
inps = tokenizer(df_test['in'].values.tolist()[4000:4500],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 69]), 9)

In [14]:
inps = tokenizer(df_test['in'].values.tolist()[4500:5000],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([1000, 64]), 10)

In [15]:
inps = tokenizer(df_test['in'].values.tolist()[5000:],
    padding=True, truncation=True, return_tensors="tf")
outs = model.generate(input_ids=inps['input_ids'],
    do_sample=True, attention_mask=inps['attention_mask'], max_length=512,
    top_k=250, top_p=0.99, early_stopping=True, num_return_sequences=preds_returned)
full_outs.append(outs)
outs.shape, len(full_outs)

(TensorShape([354, 76]), 11)

In [16]:
# putting everything back together
longest_out = max([x.shape[1] for x in full_outs])
np_outs = None
for batch in full_outs:
    padded = np.pad(batch.numpy(), ((0, 0), (0, longest_out-batch.shape[1])), 'constant')
    if np_outs is None:
        np_outs = padded
    else:
        np_outs = np.concatenate((np_outs, padded), axis=0)
sentence_outputs = np.reshape(
    [x.split() for x in tokenizer.batch_decode(np_outs, skip_special_tokens=True)],
    (df_test.shape[0], -1))
to_df = [list(x) for x in sentence_outputs] # allows us to put all generated sentences in 1 column in list form
df_test['out'] = to_df
df_test.head()

/usr/local/anaconda3/envs/cs-7643/lib/python3.7/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


,in,expected,sentence_inputs,to_model,encoding,input_ids,attention_mask,out
0,"PCCW's chief operating officer, Mike Butcher, ...",Current Chief Operating Officer Mike Butcher a...,"[PCCW's, chief, operating, officer,, Mike, But...","paraphrase: PCCW's chief operating officer, Mi...","[input_ids, attention_mask]","((tf.Tensor(2104, shape=(), dtype=int32), tf.T...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[Mike, Butcher,, PCCW, Chief, Operating, Offi..."
1,The world's two largest automakers said their ...,Domestic sales at both GM and No. 2 Ford Motor...,"[The, world's, two, largest, automakers, said,...",paraphrase: The world's two largest automakers...,"[input_ids, attention_mask]","((tf.Tensor(37, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[Expatriate, American, shopping, numbers, fel..."
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...,"[According, to, the, federal, Centers, for, Di...",paraphrase: According to the federal Centers f...,"[input_ids, attention_mask]","((tf.Tensor(2150, shape=(), dtype=int32), tf.T...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[In, 2002,, there, were, 19, reported, cases,..."
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...,"[A, tropical, storm, rapidly, developed, in, t...",paraphrase: A tropical storm rapidly developed...,"[input_ids, attention_mask]","((tf.Tensor(71, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[A, tropical, storm, in, the, Gulf, of, Mexic..."
4,The company didn't detail the costs of the rep...,But company officials expect the costs of the ...,"[The, company, didn't, detail, the, costs, of,...",paraphrase: The company didn't detail the cost...,"[input_ids, attention_mask]","((tf.Tensor(37, shape=(), dtype=int32), tf.Ten...","((tf.Tensor(1, shape=(), dtype=int32), tf.Tens...","[[The, company, doesn't, detail, the, costs, o..."


In [24]:
sentence_inputs = [x.split() for x in df_test['in'].values.tolist()]
sentence_expected = [x.split() for x in df_test['expected'].values.tolist()]
sentence_outputs = df_test['out'].values.tolist()
input_bleu = corpus_bleu(sentence_outputs, sentence_inputs)
exp_bleu = corpus_bleu(sentence_outputs, sentence_expected)
print(input_bleu, exp_bleu)

0.638831997077511 0.41971248401290545


In [22]:
sentence_inputs[0]

"PCCW's chief operating officer, Mike Butcher, and Alex Arena, the chief financial officer, will report directly to Mr So."

In [18]:
df_test.to_csv('./data/combined_test_with_preds.tsv', sep='\t')